In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [2]:
data = pd.read_csv('Dashboard_Main_data.csv', engine= 'python')

In [3]:
data['year'] = pd.DatetimeIndex(data['Date']).year

In [4]:
dropdown_option=[{'label': i, 'value': i} for i in data.Category.unique()]

In [5]:
app = dash.Dash()

In [6]:
app.layout = html.Div(children=
                      [
                          html.H1("Ecommerce Website Sales Dashboard for 2018 - 20",
                                  style={'height':'20%','width':'50%','display':'inline-block', "border":'1px black solid'}),
                          dcc.Dropdown(id = 'Category_Dropdown',options = dropdown_option,
                                       placeholder = 'Please select a category',value = 'Mobile',
                                       style={'height':'5%','width':'45%','display':'inline-block'},
                                       #multi = True
                                      ),
                          dcc.Slider(
                              id='year_slider',
                              min=data['year'].min(),
                              max=data['year'].max(),
                              value=data['year'].min(),
                              marks={str(year): str(year) for year in data['year'].unique()},
                              step=None),
                          html.H1("asdfghjklwertyuiopxcvbnm",
                                  style={'height':'20%','width':'50%','display':'block', "border":'1px black solid'}),
                          
                          dcc.Graph(id = 'Bar_Graph1', 
                                    style={'height':'40%','width':'33%','display':'inline-block', "border":"1px black solid"}
                          ),
                          dcc.Graph(id = 'Gauge_Chart1', 
                                    style={'height':'40%','width':'33%','display':'inline-block', "border":"1px black solid"}
                          ),
                          dcc.Graph(id = 'Gauge_Chart2', 
                                    style={'height':'40%','width':'33%','display':'inline-block', "border":"1px black solid"}   
                          ),
                          dcc.Graph(id = 'Area_Graph1', 
                                    style={'height':'40%','width':'100%','display':'inline-block', "border":"1px black solid"}
                          )
                      ]
                     )

In [7]:
@app.callback(
    [
     Output('Bar_Graph1','figure'),
     Output('Gauge_Chart1','figure'),
     Output('Gauge_Chart2','figure'),
     Output('Area_Graph1','figure')
    ],
    [Input('Category_Dropdown','value'),
     Input('year_slider','value')
    ]
)

def update_charts(dropdown_value, slider_value):
    filter_data = data[(data['Category']==dropdown_value)&(data['year']==slider_value)]
    
    ########Pivot for bar chart###############
    
    agg_data1=pd.pivot_table(filter_data,index=['Category'],values=['Sales'],aggfunc = np.sum)
    flat1 = pd.DataFrame(agg_data1.to_records())
    fig1 = px.bar(flat1, x = 'Category', y='Sales')
    fig1.update_layout(transition_duration=500)
    
    #######Gauge chart 1##########################
    fig2 = go.Figure(go.Indicator(
                                    domain = {'x': [0, 1], 'y': [0, 1]},
                                    value = filter_data['Rating'].mean(), 
                                    mode = "gauge+number",
                                    title = {'text': "Average Of Rating"},delta = {'reference': 0},
                                    gauge = {'axis': {'range': [None, 5]}}
                                )
                    )
    fig2.update_layout(transition_duration=500)
    
    ####Gauge chart 2 ###############################
    fig3 = go.Figure(go.Indicator(
                                    domain = {'x': [0, 1], 'y': [0, 1]},
                                    value = filter_data['Discount'].mean(), 
                                    mode = "gauge+number",
                                    title = {'text': "Average Of Discount"},delta = {'reference': 0},
                                    gauge = {'axis': {'range': [None, 15]}}
                                )
                    )
    fig3.update_layout(transition_duration=500)
    
    #######Pivot for area chart##################
    agg_data2=pd.pivot_table(filter_data,index=['Product Name'],values=['Sales'],aggfunc = np.sum)
    flat2 = pd.DataFrame(agg_data2.to_records())
    fig4 = go.Figure(go.Scatter(name = 'Sales By Product Name',x=flat2['Product Name'], y=flat2['Sales'], fill='tozeroy', 
                              text=list(flat2['Sales']), textposition="top center", mode="lines+markers+text" ))
    fig4.update_layout(transition_duration=500)
    
    return [fig1,fig2, fig3,fig4]

In [8]:
##Command to run the app

if __name__ == '__main__':
    app.run_server(debug=False)
    

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Aug/2020 10:19:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Aug/2020 10:19:38] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [29/Aug/2020 10:19:38] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [29/Aug/2020 10:19:39] "POST /_dash-update-component HTTP/1.1" 200 -
